<a href="https://colab.research.google.com/github/stud0000212941/laba_8/blob/main/new_laba8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Требуется написать приложение, реализующее следующий функционал:
1.	Извлечь данные по таблицам Бомбардиры, Ассистенты, Штрафники. (https://soccer365.ru/competitions/13/)
Внимание! Один и тот же игрок может присутствовать в нескольких таблицах.
2.	Собрать из этих данных датафрейм 


Получить по датафрейму следующую аналитику:
1)	Первая тройка команд по числу забитых голов с выводом их числа.
2)	Первая тройка команд по числу желтых карточек.
3)	Список игроков, которые участвовали не во всех играх своей команды. Число игр команды определить по максимальному числу матчей ее игроков.
4)	Доля пенальти по отношению к числу голов для каждой команды.
5)	Корреляция числа голов с количеством очков команды. Очки взять из первой таблицы на странице по рейтингу команд.


In [ ]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
BASE_URL = 'https://soccer365.ru/competitions/13/'

In [ ]:
clearing = ["<", ">" , "/a", "title=\"", "\""]
base_url = requests.get(BASE_URL)
html = BeautifulSoup(base_url.content, "html.parser")
stat = html.select("table.comp_table_v2")
search_value = re.compile(r"(\>[0-9][0-9]\<|\>[0-9]\<|\>\s\<)")
search_team = re.compile(r"(title=\"\S*\"|title=\"\S*\s\S*\"|title=\"\S*\s\S*\s\S*\")")
search_name = re.compile(r"(>\w*\s\w*</a|>\w*</a)")
search = {0 : search_name, 1 : search_team}

In [ ]:
# Настройки 1
columns=["Name", "Team", "Class", "Match", "gol", "Pen"]
num = 4
table_num = 0
Class = "Бомбардир"

In [ ]:
# настройки 2
columns=["Name", "Team", "Class", "Match", "Asist"]
num = 3
table_num = 1
Class = "Ассистент"

In [ ]:
# настройки 3
columns=["Name", "Class", "Team", "Match", "SCHTRAVBal", "jel", "jel_po2", "krasn"]
num = 6
table_num = 2
Class = "Штрафник"

In [ ]:
content = stat[table_num].select("td")
content_value = stat[table_num].select("td.bkcenter")
i=0
base = []


for cycle in range(0, len(content), num):
    line = []


    for cycle_small in range(0,2):
        data = re.search(search[cycle_small], str(content[cycle])).group()
    
    
        for cycle_clearing in range(0,5):
            data = data.replace(clearing[cycle_clearing], "")
        line.append(data)
    line.append(Class)


    for cycle_small in range(0,(num-1)):
        value = re.search(search_value, str(content_value[i])).group()
        value = value.replace("\xa0", "0")


        for cycle_clearing in range(0,2):
            value = value.replace(clearing[cycle_clearing], "")
        i = i + 1
        line.append(int(value))
    base.append(line)


table = pd.DataFrame(base, columns = columns)
table

,Name,Class,Team,Match,SCHTRAVBal,jel,jel_po2,krasn
0,Роберто Фернандес,Динамо Москва,Штрафник,9,3,2,0,11
1,Мамаду Майга,Пари Нижний Новгород,Штрафник,9,6,1,0,14
2,Антон Швец,Ахмат,Штрафник,7,7,0,0,16
3,Андрей Егорычев,Урал,Штрафник,6,3,1,0,9
4,Кайо,Краснодар,Штрафник,6,3,1,0,11
5,Мингиян Бевеев,Урал,Штрафник,6,3,1,0,14
6,Артём Самсонов,Торпедо Москва,Штрафник,6,6,0,0,15
7,Олег Дмитриев,Факел,Штрафник,6,3,0,1,16
8,Тимур Сулейманов,Пари Нижний Новгород,Штрафник,6,3,1,0,16
9,Александр Соболев,Спартак,Штрафник,6,6,0,0,16


In [ ]:
stat_scores = html.select("table.stngs")
content = stat_scores[0].select("tr")
base = []


for isk in range(1, int(len(content))):
    line = []
    team = re.search(search_name, str(content[isk].select("a"))).group()
    

    for cycle_clearing in range(0,5):
        team = team.replace(clearing[cycle_clearing], "")
    line.append(team)
    

    data = content[isk].select("td.ctr")
    value = re.search(search_value, str(data[4])).group()
    value = value.replace("\xa0", "0")
    

    for cycle_clearing in range(0,2):
        value = value.replace(clearing[cycle_clearing], "")
    line.append(value)


    value_scores =data[7].select("b")
    value_scores = re.search(search_value, str(value_scores)).group()
    value_scores = value_scores.replace("\xa0", "0")

    
    for cycle_clearing in range(0,2):
        value_scores = value_scores.replace(clearing[cycle_clearing], "")
    line.append(value_scores)
    base.append(line)


table_scores = pd.DataFrame(base, columns=["Team", "zabit", "Bal"])
table_scores

,Team,zabit,Bal
0,Зенит,47,42
1,Спартак,40,36
2,Ростов,32,35
3,Динамо Москва,33,29
4,ЦСКА Москва,30,29
5,Ахмат,27,26
6,Оренбург,33,25
7,Краснодар,31,25
8,ПФК Сочи,27,25
9,Урал,21,21
